# Deep Learning Convenience Functions

> This notebook contains convenience functions to aid in modeling. 

In [ ]:
#| default_exp dlfn

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export

def calc_cs(x # numeric array
           ): 
    "Calculate nan mean and nan std of an array. Returned as list"
    import numpy as np
    return [np.nanmean(x, axis = 0), np.nanstd(x, axis = 0)]

In [ ]:
#| export

def apply_cs(xs, 
             cs_dict_entry # list of length 2 containing mean and s
            ): return ((xs - cs_dict_entry[0]) / cs_dict_entry[1])

In [ ]:
#| export

def reverse_cs(xs, cs_dict_entry): return (cs_dict_entry[1] * xs) + cs_dict_entry[0]

In [ ]:
#| export

def train_loop(dataloader, model, loss_fn, optimizer, silent = False):
#     import numpy as np
#     import pandas as pd
    import torch
    from torch.utils.data import Dataset
    from torch.utils.data import DataLoader
#     from torch import nn
    size = len(dataloader.dataset)
    for batch, (xs_i, y_i) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(xs_i)
        loss = loss_fn(pred, y_i) # <----------------------------------------

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(y_i) # <----------------
            if not silent:
                print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


In [ ]:
#| export

def train_error(dataloader, model, loss_fn, silent = False):
#     import numpy as np
#     import pandas as pd
    import torch
    from torch.utils.data import Dataset
    from torch.utils.data import DataLoader
#     from torch import nn
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    train_loss = 0

    with torch.no_grad():
        for xs_i, y_i in dataloader:
            pred = model(xs_i)
            train_loss += loss_fn(pred, y_i).item() # <----------------------
            
    train_loss /= num_batches
    return(train_loss) 

In [ ]:
#| export

def test_loop(dataloader, model, loss_fn, silent = False):
#     import numpy as np
#     import pandas as pd
    import torch
    from torch.utils.data import Dataset
    from torch.utils.data import DataLoader
#     from torch import nn

    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss = 0

    with torch.no_grad():
        for xs_i, y_i in dataloader:
            pred = model(xs_i)
            test_loss += loss_fn(pred, y_i).item() # <-----------------------

    test_loss /= num_batches
    if not silent:
        print(f"Test Error: Avg loss: {test_loss:>8f}")
    return(test_loss) 

In [ ]:
#| export

def yhat_loop(dataloader, model):
    import numpy as np
    import pandas as pd
    import torch
#     from torch.utils.data import Dataset
#     from torch.utils.data import DataLoader
#     from torch import nn

    
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    
    y_true = np.array([])
    y_pred = np.array([])
    
    with torch.no_grad():
        for xs_i, y_i in dataloader:
            yhat_i = model(xs_i)
            y_pred = np.append(y_pred, np.array(yhat_i.cpu()))
            y_true = np.append(y_true, np.array(y_i.cpu()))
    
    out = np.concatenate([y_true[:, None], y_pred[:, None]], axis = 1) 
    out = pd.DataFrame(out, columns = ['y_true', 'y_pred'])
    return(out)

In [ ]:
#| export

def train_nn(
    cache_path,
    training_dataloader,
    testing_dataloader,
    model,
    learning_rate = 1e-3,
    batch_size = 64,
    epochs = 500,
    model_prefix = 'model'
):
    import numpy as np
    import pandas as pd
    import torch
#     from torch.utils.data import Dataset
#     from torch.utils.data import DataLoader
    from torch import nn
    from tqdm import tqdm
    
    # Initialize the loss function
    loss_fn = nn.MSELoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

    loss_df = pd.DataFrame([i for i in range(epochs)], columns = ['Epoch'])
    loss_df['TrainMSE'] = np.nan
    loss_df['TestMSE']  = np.nan

    for t in tqdm(range(epochs)):        
        # print(f"Epoch {t+1}\n-------------------------------")
        train_loop(training_dataloader, model, loss_fn, optimizer, silent = True)

        loss_df.loc[loss_df.index == t, 'TrainMSE'
                   ] = train_error(training_dataloader, model, loss_fn, silent = True)
        
        loss_df.loc[loss_df.index == t, 'TestMSE'
                   ] = test_loop(testing_dataloader, model, loss_fn, silent = True)
        
        if (t+1)%5 == 0: # Cache in case training is interupted. 
            # print(loss_df.loc[loss_df.index == t, ['TrainMSE', 'TestMSE']])
            torch.save(model.state_dict(), 
                       cache_path+'/'+model_prefix+'_'+str(t)+'_'+str(epochs)+'.pt') # convention is to use .pt or .pth
        
    return([model, loss_df])

In [ ]:
#| export

def estimate_iterations(sec_per_it = 161):
    import math
    import pandas as pd
    hours = [1, 2, 4, 8, 12, 24]
    res = pd.DataFrame(zip(hours, 
    [math.floor(
        ((i)*(60*60))/sec_per_it
    ) for i in hours]), columns = ['Hours', 'Iterations'])
    return(res)

In [ ]:
#| export


import torch
from torch.utils.data import Dataset
class ACGTDataset(Dataset): # for any G containing matix with many (phno) to one (geno)
    def __init__(self, 
                 y, 
                 G, # not on gpu
                 idx_original,
                 idx_lookup,
                 transform = None, target_transform = None,
                 use_gpu_num = 0,
                 device = 'cuda',
                 **kwargs 
                ):

        self.device = device
        self.y = y 
        self.G = G
        self.idx_original = idx_original
        self.idx_lookup = idx_lookup
        self.transform = transform
        self.target_transform = target_transform    
        
    def __len__(self):
        return len(self.y)
    
    def __getitem__(self, idx):
        y_idx =self.y[idx]
            
        #                 |array containing correct index in deduplicated g 
        #                 |               index in phno    
        uniq_g_idx = self.idx_lookup[self.idx_original[idx], 1]
        g_idx = self.G[uniq_g_idx, :, :]
        
        # send all to gpu        
        if (self.device != 'cpu'):
            if y_idx.device.type == 'cpu':
                y_idx = y_idx.to(self.device) 
                
            if g_idx.device.type == 'cpu':
                g_idx = g_idx.to(self.device) 
        
        
        if self.transform:
            g_idx = self.transform(g_idx)
            
        if self.target_transform:
            y_idx = self.transform(y_idx)
        return g_idx, y_idx

In [ ]:
# #| export

# def train_loop3(dataloader, model, loss_fn, optimizer, silent = False):
#     "This is a version of train_loop which concatenates three ys."
#     import torch
#     from torch.utils.data import Dataset
#     from torch.utils.data import DataLoader
#     size = len(dataloader.dataset)
#     for batch, (xs_i, y1_i, y2_i, y3_i) in enumerate(dataloader):
#         # Compute prediction and loss
#         pred = model(xs_i)
#         loss = loss_fn(pred, torch.concat([y1_i, y2_i, y3_i], axis = 1)) # <----------------------------------------

#         # Backpropagation
#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()

#         if batch % 100 == 0:
#             loss, current = loss.item(), batch * len(y1_i) # <----------------
#             if not silent:
#                 print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

                

In [ ]:
# #| export

# def train_error3(dataloader, model, loss_fn, silent = False):
#     import torch
#     from torch.utils.data import Dataset
#     from torch.utils.data import DataLoader
#     size = len(dataloader.dataset)
#     num_batches = len(dataloader)
#     train_loss = 0

#     with torch.no_grad():
#         for xs_i, y1_i, y2_i, y3_i in dataloader:
#             pred = model(xs_i)
#             train_loss += loss_fn(pred, torch.concat([y1_i, y2_i, y3_i], axis = 1)).item() # <----------------------
            
#     train_loss /= num_batches
#     return(train_loss) 

In [ ]:
# #| export

# def test_loop3(dataloader, model, loss_fn, silent = False):
#     import torch
#     from torch.utils.data import Dataset
#     from torch.utils.data import DataLoader

#     size = len(dataloader.dataset)
#     num_batches = len(dataloader)
#     test_loss = 0

#     with torch.no_grad():
#         for xs_i, y1_i, y2_i, y3_i in dataloader:
#             pred = model(xs_i)
#             test_loss += loss_fn(pred, torch.concat([y1_i, y2_i, y3_i], axis = 1)).item() # <-----------------------

#     test_loss /= num_batches
#     if not silent:
#         print(f"Test Error: Avg loss: {test_loss:>8f}")
#     return(test_loss) 

In [ ]:
# #| export

# def yhat_loop3(dataloader, model):
#     "Version of yhat_loop that returns 3 ys"
#     import numpy as np
#     import pandas as pd
#     import torch
#     size = len(dataloader.dataset)
#     num_batches = len(dataloader)
    
#     first_loop = True
    
#     with torch.no_grad():
#         for xs_i, y1_i, y2_i, y3_i in dataloader:
#             yhat_i = model(xs_i)
#             y_i = torch.concat([y1_i, y2_i, y3_i], axis = 1) # <-----------------------

#             if first_loop:
#                 y_pred = np.array(yhat_i.cpu())
#                 y_true = np.array(y_i.cpu())
#                 first_loop = False
#             else:            
#                 y_pred = np.concatenate([y_pred, np.array(yhat_i.cpu())])
#                 y_true = np.concatenate([y_true, np.array(y_i.cpu())])
                
#     out = np.concatenate([y_true[:, :], y_pred[:, :]], axis = 1) 
#     out = pd.DataFrame(out, columns = ['y1_true', 'y2_true', 'y3_true', 'y1_pred', 'y2_pred', 'y3_pred'])
#     return(out)

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()